<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/BILSTM%2BGLOVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
device_name = tf.test.gpu_device_name()
print(device_name)
# df_train = pd.read_csv('/content/SA-emotion-train_val_test_clean_tweets.csv')
df = pd.read_csv('/content/SA-emotion-train_val_test_clean_tweetsekphrasis.csv', sep='\t')
# df_train = df_train.sample(frac=1)
df = df.sample(frac=1)

In [ ]:
drive.mount('/content/drive')

In [ ]:
nltk.download('all')

In [ ]:
df.head()

In [ ]:
df.emotion.unique()

In [ ]:
category_dict = {0:[1,0,0,0,0],1:[0,1,0,0,0],2:[0,0,1,0,0],3:[0,0,0,1,0],4:[0,0,0,0,1]}
data_tweet = [x.lower().split() for x in df['tweets']]
data_cat = np.array([category_dict[x] for x in df['emotion']])

data_tweet = data_tweet

In [ ]:
X_train_data, X_test_data, y_train_data, y_test_data = train_test_split(data_tweet, data_cat, test_size=0.2, random_state=42)
X_train_data, X_val_data, y_train_data, y_val_data = train_test_split(X_train_data, y_train_data, test_size=0.2, random_state=42)

In [ ]:
print(len(X_train))
print(len(X_test))
print(len(X_val))

#Glove

In [ ]:
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [ ]:
batch_size = 500
Max_input_size = max([len(x) for x in data_tweet])
mul_factor=1.5

In [ ]:
X_train = [nltk.pos_tag(x) for x in X_train_data]
X_test = [nltk.pos_tag(x) for x in X_test_data]
X_val = [nltk.pos_tag(x) for x in X_val_data]


# Multiplication

In [ ]:
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='NN':
                tweet[z]=((tweet[z][0]*tweet[i][0])*mul_factor/np.sqrt(tweet[i][0].dot(tweet[i][0])),tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]*tweet[i][0])*mul_factor/np.sqrt(tweet[i][0].dot(tweet[i][0])),tweet[z][1])      
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [ ]:
epochs=20
Embedding_size=100
model_mul = tf.keras.Sequential()
model_mul.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_mul.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_mul.add(tf.keras.layers.Dropout(0.2))
model_mul.add(tf.keras.layers.Dense(64,activation='relu'))
model_mul.add(tf.keras.layers.Dropout(0.2))
model_mul.add(tf.keras.layers.Dense(5, activation='softmax'))
model_mul.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])

validation_generator = glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size,mul_factor)
history_mul = model_mul.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs,validation_steps=2)
history_mul_eval = model_mul.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))

In [ ]:
print(history_mul_eval)

#Normal

In [ ]:
X_train = X_train_data
X_test = X_test_data
X_val = X_val_data
y_train = y_train_data
y_test = y_test_data
y_val = y_val_data


In [ ]:
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size=100):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([vocab[i] for i in x if i in vocab.keys()]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [ ]:
epochs=27
Embedding_size=200
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

# validation_generator = glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size,mul_factor)
history_mul = model.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history_mul_eval = model.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))

In [ ]:
epochs=27
Embedding_size=200
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

# validation_generator = glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size,mul_factor)
train_generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size)
val_generator = glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_mul = model.fit(train_generator,validation_data=val_generator, validation_steps=100,steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history_mul_eval = model.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))

#mult

In [ ]:
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([np.array(vocab[i])*mul_factor if (i in vocab.keys() and (nltk.pos_tag(nltk.word_tokenize(i))[0][1]=='JJ' or nltk.pos_tag(nltk.word_tokenize(i))[0][1]=='NN')) else np.array(vocab[i]) for i in x if i in vocab.keys()]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [ ]:
epochs=27
Embedding_size=200
model_mul = tf.keras.Sequential()
model_mul.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_mul.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_mul.add(tf.keras.layers.Dropout(0.2))
model_mul.add(tf.keras.layers.Dense(64,activation='relu'))
model_mul.add(tf.keras.layers.Dropout(0.2))
model_mul.add(tf.keras.layers.Dense(5, activation='softmax'))
model_mul.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

# validation_generator = glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size,mul_factor)
train_generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size)
val_generator = glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_mul = model_mul.fit(train_generator,validation_data=val_generator, validation_steps=100,steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history_mul_eval = model_mul.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))

#posneg

In [ ]:
X_train_posneg = X_train_data
X_test_posneg = X_test_data
X_val_posneg = X_val_data
y_train_posneg = y_train_data
y_test_posneg = y_test_data
y_val_posneg = y_val_data


In [ ]:
epochs=27
Embedding_size=200
model_posneg = tf.keras.Sequential()
model_posneg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_posneg.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_posneg.add(tf.keras.layers.Dropout(0.2))
model_posneg.add(tf.keras.layers.Dense(64,activation='relu'))
model_posneg.add(tf.keras.layers.Dropout(0.2))
model_posneg.add(tf.keras.layers.Dense(5, activation='softmax'))
model_posneg.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

# validation_generator = glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size,mul_factor)
train_generator = glove_gen(vocab_glove,X_train_posneg,y_train_posneg,batch_size,Max_input_size,Embedding_size)
val_generator = glove_gen(vocab_glove,X_val_posneg,y_val_posneg,batch_size,Max_input_size,Embedding_size)
history_mul = model_posneg.fit(train_generator,validation_data=val_generator, validation_steps=100,steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history_mul_eval = model_posneg.evaluate(x = glove_gen(vocab_glove,X_test_posneg,y_test_posneg,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))